In [13]:
from klampt import vis
from klampt import WorldModel
from klampt.plan import robotplanning
from klampt.model import ik

In [2]:
# load robot arm
world = WorldModel()
world.readFile("TRINA_Model/urdf/TRINA_Defender.urdf")

# save robot reference
robot = world.robot(0)

q_home = get_home_config()
robot.setConfig(q_home)

# torso_link: 25
# left_base_link: 6
# right_base_link: 15
robot.enableSelfCollision(6, 25, False)
robot.enableSelfCollision(15, 25, False)

# right_EE_link: 21
# right_shielf_link: 23
robot.enableSelfCollision(21, 23, False)

vis.add("world", world)
vis.show()

q_goal = robot.getConfig()
q_goal[8] -= 1.0
planner = robotplanning.plan_to_config(world, robot, q_goal)
# planner.getPath()

KlamptWidgetAdaptor(scene={'metadata': {'version': 4.4, 'type': 'Object', 'fullscene': True, 'generator': 'Kla…

In [33]:
for i in range(robot.numLinks()):
    l = robot.link(i)
    print("link", i, l.getName())

link 0 base0
link 1 base1
link 2 base2
link 3 base3
link 4 base4
link 5 base_link
link 6 left_base_link
link 7 left_shoulder_link
link 8 left_upperarm_link
link 9 left_forearm_link
link 10 left_wrist1_link
link 11 left_wrist2_link
link 12 left_EE_link
link 13 left_tool_link
link 14 left_wrist2_camera_link
link 15 right_base_link
link 16 right_shoulder_link
link 17 right_upperarm_link
link 18 right_forearm_link
link 19 right_wrist1_link
link 20 right_wrist2_link
link 21 right_EE_link
link 22 right_tool_link
link 23 right_shield_link
link 24 right_wrist2_camera_link
link 25 torso_link
link 26 head_base_link
link 27 head_neck1_link
link 28 head_neck2_link


In [25]:
left_wrist = robot.link('left_wrist1_link')
world1 = left_wrist.getWorldPosition([0, 0, 0])
world2 = left_wrist.getWorldPosition([0, 0, 1])
world1[0] += 0.01
world2[0] += 0.01
print("world1:", world1)
print("world2:", world2)

left_obj = ik.objective(left_wrist, local=[[0, 0, 0],[0, 0, 1]], 
                        world=[world1,world2])

world1: [0.8091326711266446, 0.6677404394773893, 1.1065313867378699]
world2: [1.0220910862859285, -0.023146205860625324, 0.41564474139984453]


In [35]:
print("before config:", robot.getConfig())
ik.solve_nearby(left_obj, 0.1,
                iters=1000, tol=1e-3, activeDofs=None,
                numRestarts = 0, feasibilityCheck=None)
print("after config:", robot.getConfig())

before config: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.3515625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
after config: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.3515625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [36]:
from klampt.io import resource

localpt = [0, 0, 0]
worldpt = [1, 1, 1]

(save,value) = resource.edit("Local point",localpt,type="Point",frame=left_wrist)
if save:
    localpt = value
(save,value) = resource.edit("World point",worldpt,type="Point",frame=None)
if save:
    worldpt = value
obj = ik.objective(left_wrist,local=localpt,world=worldpt)

vis.init(): Trying to reset from backend IPython to one of ['PyQt']


AssertionError: Invalid window id

In [2]:
from klampt.io import load
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
pcd = o3d.io.read_point_cloud('Dataset/torso_pcd.ply')
o3d.io.write_point_cloud('Dataset/torso_pcd.pcd', pcd)

True

In [10]:
pcd = load('auto', 'Dataset/torso_pcd.pcd')
pcd.type()

'PointCloud'

In [17]:
help(vis.add)

Help on function add in module klampt.vis.visualization:

add(name, item, keepAppearance=False, **kwargs)
    Adds an item to the visualization.
    
    Args:
        name (str): a unique identifier.  If an item with the same name already
            exists, it will no longer be shown. 
        keepAppearance (bool, optional): if True, then if there was an item that
            had the same name, the prior item's appearance will be kept.
        kwargs: key-value pairs to be added into the attributes dictionary.  e.g.
            vis.add("geom",geometry,color=[1,0,0,1]) adds a geometry while setting
            its color to red.



In [21]:
from klampt.model import geometry

box = geometry.box(0.01, 0.01, 0.01)